Restarted py39 (Python 3.9.-1)

In [1]:
from all_funx import *

In [2]:
# import dfs from 1_clean_excel_summary.py
path  = "/Users/mariapalafox/Desktop/BOLDcaution/WGS/5-annotation/3-interpret/"

In [3]:
indel12 = pd.read_csv('1_clean_vep_output-indel-cad12-v2.csv', low_memory=False)
snp12 = pd.read_csv('1_clean_vep_output-snp-cad12-v2.csv', low_memory=False)
print(indel12.shape)
print(snp12.shape)

(546104, 80)
(2950097, 85)


In [4]:
indel63 = pd.read_csv('1_clean_vep_output-indel-cad63-v2.csv', low_memory=False)
snp63 = pd.read_csv('1_clean_vep_output-snp-cad63-v2.csv', low_memory=False)
print(indel63.shape)
print(snp63.shape)

(562353, 80)
(3138661, 85)


In [5]:
cad12 = pd.concat([indel12, snp12], axis=0)
cad63 = pd.concat([indel63, snp63], axis=0)
print(cad12.shape)
print(cad63.shape)

(3496201, 85)
(3701014, 85)


In [6]:
# create a list of unique 'Uploaded_variation' in both dataframes
cad12_variations = cad12['Uploaded_variation'].unique()
cad63_variations = cad63['Uploaded_variation'].unique()
# create a separate list for variations that are exclusively in cad12
cad12_exclusive_variations = list(set(cad12_variations) - set(cad63_variations))
# create a separate list for variations that are exclusively in cad63
cad63_exclusive_variations = list(set(cad63_variations) - set(cad12_variations))
# create a separate list for variations that are in both cad12 and cad63
common_variations = list(set(cad12_variations) & set(cad63_variations))
print( len(cad12_exclusive_variations), len(cad63_exclusive_variations), len(common_variations)) # 1525780 1730593 1970421

1525780 1730593 1970421


In [7]:
# Optimize the code by avoiding unnecessary function calls and repeated operations
cad12['var_in_both_lines'] = cad12['Uploaded_variation'].isin(common_variations)
cad12['var_exclusive_btw_lines'] = cad12['Uploaded_variation'].isin(cad12_exclusive_variations)
cad63['var_in_both_lines'] = cad63['Uploaded_variation'].isin(common_variations)
cad63['var_exclusive_btw_lines'] = cad63['Uploaded_variation'].isin(cad63_exclusive_variations)


# create a list of unique 'SYMBOL' in both dataframes
cad12_genes = cad12['SYMBOL'].unique()
cad63_genes = cad63['SYMBOL'].unique()
# create a separate list for SYMBOL that are exclusively in cad12
cad12_exclusive = list(set(cad12_genes) - set(cad63_genes))
# create a separate list for SYMBOL that are exclusively in cad63
cad63_exclusive = list(set(cad63_genes) - set(cad12_genes))
# create a separate list for SYMBOL that are in both cad12 and cad63
common_genes = list(set(cad12_genes) & set(cad63_genes))
cad12['gene_in_both_lines'] = cad12['SYMBOL'].isin(common_genes)
cad12['gene_exclusive_btw_lines'] = cad12['SYMBOL'].isin(cad12_exclusive)
cad63['gene_in_both_lines'] = cad63['SYMBOL'].isin(common_genes)
cad63['gene_exclusive_btw_lines'] = cad63['SYMBOL'].isin(cad63_exclusive)
print( len(cad12_exclusive), len(cad63_exclusive), len(common_genes)) #809 743 28137

809 743 28137


In [8]:
save_describe(cad12, '4-cad12')
save_describe(cad63, '4-cad63')

In [9]:
print(cad12.columns)
print(cad12.shape)
"""
['Uploaded_variation', 'Location', 'Allele', 'Gene', 'Feature', 'Feature_type', 'Consequence', 'cDNA_position', 'CDS_position', 'Protein_position', 'Amino_acids', 'Codons', 'Existing_variation', 'IND', 'ZYG', 'REF_ALLELE', 'IMPACT', 'DISTANCE', 'STRAND', 'FLAGS', 'VARIANT_CLASS', 'SYMBOL', 'SYMBOL_SOURCE', 'HGNC_ID', 'BIOTYPE', 'CANONICAL', 'MANE_SELECT', 'MANE_PLUS_CLINICAL', 'TSL', 'APPRIS', 'CCDS', 'ENSP', 'SWISSPROT', 'TREMBL', 'UNIPARC', 'UNIPROT_ISOFORM', 'REFSEQ_MATCH', 'SOURCE', 'REFSEQ_OFFSET', 'GIVEN_REF', 'USED_REF', 'BAM_EDIT', 'GENE_PHENO', 'SIFT', 'PolyPhen', 'EXON', 'INTRON', 'DOMAINS', 'miRNA', 'HGVSc', 'HGVSp', 'HGVS_OFFSET', 'gnomADg_AF', 'gnomADg_AFR_AF', 'gnomADg_AMI_AF', 'gnomADg_AMR_AF', 'gnomADg_ASJ_AF', 'gnomADg_EAS_AF', 'gnomADg_FIN_AF', 'gnomADg_MID_AF', 'gnomADg_NFE_AF', 'gnomADg_OTH_AF', 'gnomADg_SAS_AF', 'MAX_AF', 'MAX_AF_POPS', 'CLIN_SIG', 'SOMATIC', 'PHENO', 'PUBMED', 'MOTIF_NAME', 'MOTIF_POS', 'HIGH_INF_POS', 'MOTIF_SCORE_CHANGE',
       'TRANSCRIPTION_FACTORS', 'chromosome', 'chrCoordinate', 'EXONnumber', 'INTRONnumber', 'chr', 'GIVEN_REF_MATCH', 'SIFTprediction', 'SIFTscore', 'PolyPhenprediction', 'PolyPhenscore', 'keyID38', 'var_in_both_lines', 'var_exclusive_btw_lines', 'gene_in_both_lines', 'gene_exclusive_btw_lines'],
      dtype='object')
(3496201, 89)
"""

Index(['Uploaded_variation', 'Location', 'Allele', 'Gene', 'Feature', 'Feature_type', 'Consequence', 'cDNA_position', 'CDS_position', 'Protein_position', 'Amino_acids', 'Codons', 'Existing_variation', 'IND', 'ZYG', 'REF_ALLELE', 'IMPACT', 'DISTANCE', 'STRAND', 'FLAGS', 'VARIANT_CLASS', 'SYMBOL', 'SYMBOL_SOURCE', 'HGNC_ID', 'BIOTYPE', 'CANONICAL', 'MANE_SELECT', 'MANE_PLUS_CLINICAL', 'TSL', 'APPRIS', 'CCDS', 'ENSP', 'SWISSPROT', 'TREMBL', 'UNIPARC', 'UNIPROT_ISOFORM', 'REFSEQ_MATCH', 'SOURCE', 'REFSEQ_OFFSET', 'GIVEN_REF', 'USED_REF', 'BAM_EDIT', 'GENE_PHENO', 'SIFT', 'PolyPhen', 'EXON', 'INTRON', 'DOMAINS', 'miRNA', 'HGVSc', 'HGVSp', 'HGVS_OFFSET', 'gnomADg_AF', 'gnomADg_AFR_AF', 'gnomADg_AMI_AF', 'gnomADg_AMR_AF', 'gnomADg_ASJ_AF', 'gnomADg_EAS_AF', 'gnomADg_FIN_AF', 'gnomADg_MID_AF', 'gnomADg_NFE_AF', 'gnomADg_OTH_AF', 'gnomADg_SAS_AF', 'MAX_AF', 'MAX_AF_POPS', 'CLIN_SIG', 'SOMATIC', 'PHENO', 'PUBMED', 'MOTIF_NAME', 'MOTIF_POS', 'HIGH_INF_POS', 'MOTIF_SCORE_CHANGE',
       'TRANSCRIP

"\n['Uploaded_variation', 'Location', 'Allele', 'Gene', 'Feature', 'Feature_type', 'Consequence', 'cDNA_position', 'CDS_position', 'Protein_position', 'Amino_acids', 'Codons', 'Existing_variation', 'IND', 'ZYG', 'REF_ALLELE', 'IMPACT', 'DISTANCE', 'STRAND', 'FLAGS', 'VARIANT_CLASS', 'SYMBOL', 'SYMBOL_SOURCE', 'HGNC_ID', 'BIOTYPE', 'CANONICAL', 'MANE_SELECT', 'MANE_PLUS_CLINICAL', 'TSL', 'APPRIS', 'CCDS', 'ENSP', 'SWISSPROT', 'TREMBL', 'UNIPARC', 'UNIPROT_ISOFORM', 'REFSEQ_MATCH', 'SOURCE', 'REFSEQ_OFFSET', 'GIVEN_REF', 'USED_REF', 'BAM_EDIT', 'GENE_PHENO', 'SIFT', 'PolyPhen', 'EXON', 'INTRON', 'DOMAINS', 'miRNA', 'HGVSc', 'HGVSp', 'HGVS_OFFSET', 'gnomADg_AF', 'gnomADg_AFR_AF', 'gnomADg_AMI_AF', 'gnomADg_AMR_AF', 'gnomADg_ASJ_AF', 'gnomADg_EAS_AF', 'gnomADg_FIN_AF', 'gnomADg_MID_AF', 'gnomADg_NFE_AF', 'gnomADg_OTH_AF', 'gnomADg_SAS_AF', 'MAX_AF', 'MAX_AF_POPS', 'CLIN_SIG', 'SOMATIC', 'PHENO', 'PUBMED', 'MOTIF_NAME', 'MOTIF_POS', 'HIGH_INF_POS', 'MOTIF_SCORE_CHANGE',\n       'TRANSCRIPTI

In [10]:
print(cad63.columns)
print(cad63.shape)
"""(3701014, 89)"""

Index(['Uploaded_variation', 'Location', 'Allele', 'Gene', 'Feature', 'Feature_type', 'Consequence', 'cDNA_position', 'CDS_position', 'Protein_position', 'Amino_acids', 'Codons', 'Existing_variation', 'IND', 'ZYG', 'REF_ALLELE', 'IMPACT', 'DISTANCE', 'STRAND', 'FLAGS', 'VARIANT_CLASS', 'SYMBOL', 'SYMBOL_SOURCE', 'HGNC_ID', 'BIOTYPE', 'CANONICAL', 'MANE_SELECT', 'MANE_PLUS_CLINICAL', 'TSL', 'APPRIS', 'CCDS', 'ENSP', 'SWISSPROT', 'TREMBL', 'UNIPARC', 'UNIPROT_ISOFORM', 'REFSEQ_MATCH', 'SOURCE', 'REFSEQ_OFFSET', 'GIVEN_REF', 'USED_REF', 'BAM_EDIT', 'GENE_PHENO', 'SIFT', 'PolyPhen', 'EXON', 'INTRON', 'DOMAINS', 'miRNA', 'HGVSc', 'HGVSp', 'HGVS_OFFSET', 'gnomADg_AF', 'gnomADg_AFR_AF', 'gnomADg_AMI_AF', 'gnomADg_AMR_AF', 'gnomADg_ASJ_AF', 'gnomADg_EAS_AF', 'gnomADg_FIN_AF', 'gnomADg_MID_AF', 'gnomADg_NFE_AF', 'gnomADg_OTH_AF', 'gnomADg_SAS_AF', 'MAX_AF', 'MAX_AF_POPS', 'CLIN_SIG', 'SOMATIC', 'PHENO', 'PUBMED', 'MOTIF_NAME', 'MOTIF_POS', 'HIGH_INF_POS', 'MOTIF_SCORE_CHANGE',
       'TRANSCRIP

'(3701014, 89)'

In [11]:
# fix column names by replacing _ with .
cad12.columns = cad12.columns.str.replace('_', '.')
cad63.columns = cad63.columns.str.replace('_', '.')


print(cad63.columns)
"""'Uploaded.variation', 'Location', 'Allele', 'Gene', 'Feature', 'Feature.type', 'Consequence', 'cDNA.position', 'CDS.position', 'Protein.position', 'Amino.acids', 'Codons', 'Existing.variation', 'IND', 'ZYG', 'REF.ALLELE', 'IMPACT', 'DISTANCE', 'STRAND', 'FLAGS', 'VARIANT.CLASS', 'SYMBOL', 'SYMBOL.SOURCE', 'HGNC.ID', 'BIOTYPE', 'CANONICAL', 'MANE.SELECT', 'MANE.PLUS.CLINICAL', 'TSL', 'APPRIS', 'CCDS', 'ENSP', 'SWISSPROT', 'TREMBL', 'UNIPARC', 'UNIPROT.ISOFORM', 'REFSEQ.MATCH', 'SOURCE', 'REFSEQ.OFFSET', 'GIVEN.REF', 'USED.REF', 'BAM.EDIT', 'GENE.PHENO', 'SIFT', 'PolyPhen', 'EXON', 'INTRON', 'DOMAINS', 'miRNA', 'HGVSc', 'HGVSp', 'HGVS.OFFSET', 'gnomADg.AF', 'gnomADg.AFR.AF', 'gnomADg.AMI.AF', 'gnomADg.AMR.AF', 'gnomADg.ASJ.AF', 'gnomADg.EAS.AF', 'gnomADg.FIN.AF', 'gnomADg.MID.AF', 'gnomADg.NFE.AF', 'gnomADg.OTH.AF', 'gnomADg.SAS.AF', 'MAX.AF', 'MAX.AF.POPS', 'CLIN.SIG', 'SOMATIC', 'PHENO', 'PUBMED', 'MOTIF.NAME', 'MOTIF.POS', 'HIGH.INF.POS', 'MOTIF.SCORE.CHANGE',
       'TRANSCRIPTION.FACTORS', 'chromosome', 'chrCoordinate', 'EXONnumber', 'INTRONnumber', 'chr', 'GIVEN.REF.MATCH', 'SIFTprediction', 'SIFTscore', 'PolyPhenprediction', 'PolyPhenscore', 'keyID38', 'var.in.both.lines', 'var.exclusive.btw.lines', 'gene.in.both.lines', 'gene.exclusive.btw.lines"""

Index(['Uploaded.variation', 'Location', 'Allele', 'Gene', 'Feature', 'Feature.type', 'Consequence', 'cDNA.position', 'CDS.position', 'Protein.position', 'Amino.acids', 'Codons', 'Existing.variation', 'IND', 'ZYG', 'REF.ALLELE', 'IMPACT', 'DISTANCE', 'STRAND', 'FLAGS', 'VARIANT.CLASS', 'SYMBOL', 'SYMBOL.SOURCE', 'HGNC.ID', 'BIOTYPE', 'CANONICAL', 'MANE.SELECT', 'MANE.PLUS.CLINICAL', 'TSL', 'APPRIS', 'CCDS', 'ENSP', 'SWISSPROT', 'TREMBL', 'UNIPARC', 'UNIPROT.ISOFORM', 'REFSEQ.MATCH', 'SOURCE', 'REFSEQ.OFFSET', 'GIVEN.REF', 'USED.REF', 'BAM.EDIT', 'GENE.PHENO', 'SIFT', 'PolyPhen', 'EXON', 'INTRON', 'DOMAINS', 'miRNA', 'HGVSc', 'HGVSp', 'HGVS.OFFSET', 'gnomADg.AF', 'gnomADg.AFR.AF', 'gnomADg.AMI.AF', 'gnomADg.AMR.AF', 'gnomADg.ASJ.AF', 'gnomADg.EAS.AF', 'gnomADg.FIN.AF', 'gnomADg.MID.AF', 'gnomADg.NFE.AF', 'gnomADg.OTH.AF', 'gnomADg.SAS.AF', 'MAX.AF', 'MAX.AF.POPS', 'CLIN.SIG', 'SOMATIC', 'PHENO', 'PUBMED', 'MOTIF.NAME', 'MOTIF.POS', 'HIGH.INF.POS', 'MOTIF.SCORE.CHANGE',
       'TRANSCRIP

"'Uploaded.variation', 'Location', 'Allele', 'Gene', 'Feature', 'Feature.type', 'Consequence', 'cDNA.position', 'CDS.position', 'Protein.position', 'Amino.acids', 'Codons', 'Existing.variation', 'IND', 'ZYG', 'REF.ALLELE', 'IMPACT', 'DISTANCE', 'STRAND', 'FLAGS', 'VARIANT.CLASS', 'SYMBOL', 'SYMBOL.SOURCE', 'HGNC.ID', 'BIOTYPE', 'CANONICAL', 'MANE.SELECT', 'MANE.PLUS.CLINICAL', 'TSL', 'APPRIS', 'CCDS', 'ENSP', 'SWISSPROT', 'TREMBL', 'UNIPARC', 'UNIPROT.ISOFORM', 'REFSEQ.MATCH', 'SOURCE', 'REFSEQ.OFFSET', 'GIVEN.REF', 'USED.REF', 'BAM.EDIT', 'GENE.PHENO', 'SIFT', 'PolyPhen', 'EXON', 'INTRON', 'DOMAINS', 'miRNA', 'HGVSc', 'HGVSp', 'HGVS.OFFSET', 'gnomADg.AF', 'gnomADg.AFR.AF', 'gnomADg.AMI.AF', 'gnomADg.AMR.AF', 'gnomADg.ASJ.AF', 'gnomADg.EAS.AF', 'gnomADg.FIN.AF', 'gnomADg.MID.AF', 'gnomADg.NFE.AF', 'gnomADg.OTH.AF', 'gnomADg.SAS.AF', 'MAX.AF', 'MAX.AF.POPS', 'CLIN.SIG', 'SOMATIC', 'PHENO', 'PUBMED', 'MOTIF.NAME', 'MOTIF.POS', 'HIGH.INF.POS', 'MOTIF.SCORE.CHANGE',\n       'TRANSCRIPTION.

In [12]:
# add MAF column to  dataframes, if MAX.AF is < 0.01 label Rare, if its less than 0.1 label less common, if its greater than or equal  to 0.1 label common, else label empty string
cad12['MAF'] = np.select([cad12['MAX.AF'] < 0.01, 
                          (cad12['MAX.AF'] >= 0.01) & (cad12['MAX.AF'] < 0.1),
                            cad12['MAX.AF'] >= 0.1], 
                            ['Rare (<1%)', 'Less common (<10%)', 'Common (>=10%)'], 
                            default = np.nan)

cad63['MAF'] = np.select([cad63['MAX.AF'] < 0.01, 
                          (cad63['MAX.AF'] >= 0.01) & (cad63['MAX.AF'] < 0.1), 
                          cad63['MAX.AF'] >= 0.1], 
                          ['Rare (<1%)', 'Less common (<10%)', 'Common (>=10%)'], 
                          default = np.nan)

In [13]:
# convert the R code into python
cad12['MAF2'] = np.select([cad12['MAX.AF'] < 0.05, 
                           cad12['MAX.AF'] >= 0.05], 
                           ['Rare (<5%)', 'Common (>=5%)'],
                            default=np.nan)

cad63['MAF2'] = np.select([cad63['MAX.AF'] < 0.05, 
                           cad63['MAX.AF'] >= 0.05], 
                           ['Rare (<5%)', 'Common (>=5%)'], 
                           default=np.nan)

In [14]:
# functionn that checks CLIN.SIG column for rows containing 'pathogenic' or 'likely pathogenic' and creates new column with 'PATHO' label. the new column should also  contain 'VUS' and 'BENIGN' labels if the row contain 'benign' for 'BENIGN' and 'uncertain significance' for 'VUS'.

def label_clin_sig(df):
    df['PATHO'] = df['CLIN.SIG'].str.contains('pathogenic|likely pathogenic', case=False)
    df['VUS'] = df['CLIN.SIG'].str.contains('uncertain_significance', case=False)
    df['BENIGN'] = df['CLIN.SIG'].str.contains('benign|likely_benign', case=False)
    return df


# new function creates one column with new labels based on rule heirarchy; if pathogenic or likely pathogenic then labeled PATHO else if uncertain_significance then VUS else if benign or likely benign then BENIGN
def label_clin_sig2(df):
    newcol = []
    for row in df.itertuples():
        if row.PATHO:
            newcol.append('PATHO')
        elif row.VUS:
            newcol.append('VUS')
        elif row.BENIGN:
            newcol.append('BENIGN')
        else:
            newcol.append(np.nan) # append nan value instead of '' emptry string
    df['ClinVar'] = newcol
    return df

In [15]:
cad12 = label_clin_sig(cad12)
cad12 = label_clin_sig2(cad12)
checkColumnValue(cad12, 'ClinVar')

,Count,count
0,NaN,3464042
1,BENIGN,31769
2,VUS,302
3,PATHO,88


In [16]:
# label ClinVar using not star reviewed system, over-estimates number of pathogenic variants
cad63 = label_clin_sig(cad63)
cad63 = label_clin_sig2(cad63)
checkColumnValue(cad63, 'ClinVar')

,Count,count
0,NaN,3667535
1,BENIGN,33038
2,VUS,330
3,PATHO,111


In [17]:
patho12 = cad12[cad12['ClinVar'] == 'PATHO']
crosstabit(cad12, 'IND', 'ClinVar', True)

ClinVar,BENIGN,NA,PATHO,VUS,Total
IND,,,,,
P732912,31769,3464042,88,302,3496201
Total,31769,3464042,88,302,3496201


In [18]:
patho63 = cad63[cad63['ClinVar'] == 'PATHO']
crosstabit(cad63, 'IND', 'ClinVar', True)

ClinVar,BENIGN,NA,PATHO,VUS,Total
IND,,,,,
P931263,33038,3667535,111,330,3701014
Total,33038,3667535,111,330,3701014


In [19]:
patho12.head()

,Uploaded.variation,Location,Allele,Gene,Feature,Feature.type,Consequence,cDNA.position,CDS.position,Protein.position,Amino.acids,Codons,Existing.variation,IND,ZYG,REF.ALLELE,IMPACT,DISTANCE,STRAND,FLAGS,VARIANT.CLASS,SYMBOL,SYMBOL.SOURCE,HGNC.ID,BIOTYPE,CANONICAL,MANE.SELECT,MANE.PLUS.CLINICAL,TSL,APPRIS,CCDS,ENSP,SWISSPROT,TREMBL,UNIPARC,UNIPROT.ISOFORM,REFSEQ.MATCH,SOURCE,REFSEQ.OFFSET,GIVEN.REF,USED.REF,BAM.EDIT,GENE.PHENO,SIFT,PolyPhen,EXON,INTRON,DOMAINS,miRNA,HGVSc,HGVSp,HGVS.OFFSET,gnomADg.AF,gnomADg.AFR.AF,gnomADg.AMI.AF,gnomADg.AMR.AF,gnomADg.ASJ.AF,gnomADg.EAS.AF,gnomADg.FIN.AF,gnomADg.MID.AF,gnomADg.NFE.AF,gnomADg.OTH.AF,gnomADg.SAS.AF,MAX.AF,MAX.AF.POPS,CLIN.SIG,SOMATIC,PHENO,PUBMED,MOTIF.NAME,MOTIF.POS,HIGH.INF.POS,MOTIF.SCORE.CHANGE,TRANSCRIPTION.FACTORS,chromosome,chrCoordinate,EXONnumber,INTRONnumber,chr,GIVEN.REF.MATCH,SIFTprediction,SIFTscore,PolyPhenprediction,PolyPhenscore,keyID38,var.in.both.lines,var.exclusive.btw.lines,gene.in.both.lines,gene.exclusive.btw.lines,MAF,MAF2,PATHO,VUS,BENIGN,ClinVar
2118,chr1_7961851_GTGCTGGACGGTGTCCCT/-,chr1:7961851-7961868,-,ENSG00000116288,ENST00000338639,Transcript,intron_variant,NaN,NaN,NaN,-,-,rs200968609,P732912,HET,GTGCTGGACGGTGTCCCT,MODIFIER,NaN,1,-,deletion,PARK7,HGNC,HGNC:16369,protein_coding,YES,NM_007262.5,-,1,P1,CCDS93.1,ENSP00000340278,Q99497.214,V9HWC2.55,UPI0000072F6F,-,-,Ensembl,-,GTGCTGGACGGTGTCCCT,GTGCTGGACGGTGTCCCT,-,1,-,-,-,1/6,-,-,ENST00000338639.10:c.-24+75_-24+92del,-,-,0.139600,0.046980,0.045050,0.1133,0.11430,0.06402,0.315800,0.095540,0.186000,0.12900,0.102000,0.315800,gnomADg_FIN,"benign,pathogenic",-,1,"16240358,28873462,30863299,28932933,34306325",-,-,-,-,-,chr1,7961851,NaN,1.0,1,True,NaN,NaN,NaN,NaN,NaN,False,True,True,False,Common (>=10%),Common (>=5%),True,False,True,PATHO
163796,chr5_83104943_C/-,chr5:83104943,-,ENSG00000152422,ENST00000396027,Transcript,frameshift_variant,103,24,8,I/X,atC/at,rs869320677,P732912,HET,C,HIGH,NaN,1,-,deletion,XRCC4,HGNC,HGNC:12831,protein_coding,YES,NM_003401.5,-,5,A1,CCDS4058.1,ENSP00000379344,Q13426.199,A0A024RAL0.63,UPI0000072B0A,Q13426-2,-,Ensembl,-,C,C,-,1,-,-,2/8,-,"PDB-ENSP_mappings:1fu1.A,PDB-ENSP_mappings:1fu...",-,ENST00000396027.9:c.25del,ENSP00000379344.4:p.His9ThrfsTer8,1,0.000394,0.000072,0.000000,0.0000,0.00000,0.00000,0.000283,0.003165,0.000765,0.00000,0.000207,0.003165,gnomADg_MID,"likely_pathogenic,pathogenic",-,1,"26822949,25728776,25839420",-,-,-,-,-,chr5,83104943,2.0,NaN,5,True,NaN,NaN,NaN,NaN,NaN,False,True,True,False,Rare (<1%),Rare (<5%),True,False,False,PATHO
269741,chr9_27573524_GCCCCG/GCCCCGGCCCCGGCCCCGGCCCCGG...,chr9:27573524-27573529,GCCCCGGCCCCGGCCCCGGCCCCGGCCCCG,ENSG00000147894,ENST00000380003,Transcript,upstream_gene_variant,NaN,NaN,NaN,-,-,rs143561967,P732912,HET,GCCCCG,MODIFIER,43.0,-1,-,sequence_alteration,C9orf72,HGNC,HGNC:28337,protein_coding,YES,NM_018325.5,-,1,P1,CCDS6522.1,ENSP00000369339,Q96LT7.150,-,UPI00001D3FEF,Q96LT7-1,-,Ensembl,-,GCCCCG,GCCCCG,-,1,-,-,-,-,-,-,-,-,-,0.046780,0.071110,0.004728,0.1178,0.01102,0.13300,0.009990,0.010000,0.017550,0.03858,0.040460,0.133000,gnomADg_EAS,pathogenic,-,1,"31836585,32471232",-,-,-,-,-,chr9,27573524,NaN,NaN,9,True,NaN,NaN,NaN,NaN,NaN,False,True,True,False,Common (>=10%),Common (>=5%),True,False,False,PATHO
446035,chr17_30237267_GGGCTGCAGGGGGGATGCTGGGGGTGCAGGG...,chr17:30237267-30237309,-,ENSG00000108576,ENST00000650711,Transcript,upstream_gene_variant,NaN,NaN,NaN,-,-,rs774676466,P732912,HET,GGGCTGCAGGGGGGATGCTGGGGGTGCAGGGGAGATGCTGGGG,MODIFIER,1570.0,-1,-,deletion,SLC6A4,HGNC,HGNC:11050,protein_coding,YES,NM_001045.6,-,-,P1,CCDS11256.1,ENSP00000498537,P31645.208,-,UPI0000135493,P31645-1,-,Ensembl,-,GGGCTGCAGGGGGGATGCTGGGGGTGCAGGGGAGATGCTGGGG,GGGCTGCAGGGGGGATGCTGGGGGTGCAGGGGAGATGCTGGGG,-,1,-,-,-,-,-,-,-,-,-,0.283100,0.145300,0.199200,0.4085,0.42210,0.59490,0.277000,0.288500,0.304000,0.30200,0.466100,0.594900,gnomADg_EAS,pathogenic,-,1,"11772685,12966525,12915525,15691525,11335745,1...",-,-,-,-,-,chr17,30237267,NaN,NaN,17,True,NaN,

In [20]:
patho63.head()

,Uploaded.variation,Location,Allele,Gene,Feature,Feature.type,Consequence,cDNA.position,CDS.position,Protein.position,Amino.acids,Codons,Existing.variation,IND,ZYG,REF.ALLELE,IMPACT,DISTANCE,STRAND,FLAGS,VARIANT.CLASS,SYMBOL,SYMBOL.SOURCE,HGNC.ID,BIOTYPE,CANONICAL,MANE.SELECT,MANE.PLUS.CLINICAL,TSL,APPRIS,CCDS,ENSP,SWISSPROT,TREMBL,UNIPARC,UNIPROT.ISOFORM,REFSEQ.MATCH,SOURCE,REFSEQ.OFFSET,GIVEN.REF,USED.REF,BAM.EDIT,GENE.PHENO,SIFT,PolyPhen,EXON,INTRON,DOMAINS,miRNA,HGVSc,HGVSp,HGVS.OFFSET,gnomADg.AF,gnomADg.AFR.AF,gnomADg.AMI.AF,gnomADg.AMR.AF,gnomADg.ASJ.AF,gnomADg.EAS.AF,gnomADg.FIN.AF,gnomADg.MID.AF,gnomADg.NFE.AF,gnomADg.OTH.AF,gnomADg.SAS.AF,MAX.AF,MAX.AF.POPS,CLIN.SIG,SOMATIC,PHENO,PUBMED,MOTIF.NAME,MOTIF.POS,HIGH.INF.POS,MOTIF.SCORE.CHANGE,TRANSCRIPTION.FACTORS,chromosome,chrCoordinate,EXONnumber,INTRONnumber,chr,GIVEN.REF.MATCH,SIFTprediction,SIFTscore,PolyPhenprediction,PolyPhenscore,keyID38,var.in.both.lines,var.exclusive.btw.lines,gene.in.both.lines,gene.exclusive.btw.lines,MAF,MAF2,PATHO,VUS,BENIGN,ClinVar
56517,chr2_48755617_-/GCTGCA,chr2:48755616-48755617,GCTGCA,ENSG00000138039,ENST00000294954,Transcript,inframe_insertion,108-109,55-56,19,P/LQP,ccg/cTGCAGCcg,rs71245621,P931263,HOM,-,MODERATE,NaN,-1,-,insertion,LHCGR,HGNC,HGNC:6585,protein_coding,YES,NM_000233.4,-,1,P2,CCDS1842.1,ENSP00000294954,P22888.225,-,UPI000013E1E1,P22888-1,-,Ensembl,-,-,-,-,1,-,-,1/11,-,"AFDB-ENSP_mappings:AF-P22888-F1.A,PANTHER:PTHR...",-,ENST00000294954.12:c.50_55dup,ENSP00000294954.6:p.Leu17_Gln18dup,-,0.219200,0.187100,0.358100,0.185500,0.300500,0.011390,0.188100,0.2930,0.267200,0.240700,0.118600,0.35810,gnomADg_AMI,"benign,pathogenic",-,1,"12679452,9851790,7556872,2244890",-,-,-,-,-,chr2,48755616,1.0,NaN,2,True,NaN,NaN,NaN,NaN,NaN,False,True,True,False,Common (>=10%),Common (>=5%),True,False,True,PATHO
284152,chr9_27573524_GCCCCG/-,chr9:27573524-27573529,-,ENSG00000147894,ENST00000380003,Transcript,upstream_gene_variant,NaN,NaN,NaN,-,-,rs143561967,P931263,HOM,GCCCCG,MODIFIER,43.0,-1,-,deletion,C9orf72,HGNC,HGNC:28337,protein_coding,YES,NM_018325.5,-,1,P1,CCDS6522.1,ENSP00000369339,Q96LT7.150,-,UPI00001D3FEF,Q96LT7-1,-,Ensembl,-,GCCCCG,GCCCCG,-,1,-,-,-,-,-,-,-,-,-,0.532600,0.441400,0.503600,0.547400,0.657700,0.610300,0.635300,0.4933,0.557600,0.544700,0.556100,0.65770,gnomADg_ASJ,pathogenic,-,1,"31836585,32471232",-,-,-,-,-,chr9,27573524,NaN,NaN,9,True,NaN,NaN,NaN,NaN,NaN,False,True,True,False,Common (>=10%),Common (>=5%),True,False,False,PATHO
345567,chr11_68774782_A/-,chr11:68774782,-,ENSG00000110090,ENST00000265641,Transcript,intron_variant,NaN,NaN,NaN,-,-,rs1169875761,P931263,HET,A,MODIFIER,NaN,-1,-,deletion,CPT1A,HGNC,HGNC:2328,protein_coding,YES,NM_001876.4,-,1,P1,CCDS8185.1,ENSP00000265641,P50416.203,A0A024R5F4.59,UPI000013D658,P50416-1,-,Ensembl,-,A,A,-,1,-,-,-,13/18,-,-,ENST00000265641.10:c.1575+534del,-,-,0.002580,0.000886,0.002732,0.003820,0.001892,0.000991,0.013790,0.0000,0.002541,0.001449,0.002844,0.01379,gnomADg_FIN,"uncertain_significance,pathogenic",-,1,12189492,-,-,-,-,-,chr11,68774782,NaN,13.0,11,True,NaN,NaN,NaN,NaN,NaN,False,True,True,False,Less common (<10%),Rare (<5%),True,True,False,PATHO
377988,chr12_101765329_-/T,chr12:101765328-101765329,T,ENSG00000111670,ENST00000299314,Transcript,intron_variant,NaN,NaN,NaN,-,-,rs546802775,P931263,HET,-,MODIFIER,NaN,-1,-,insertion,GNPTAB,HGNC,HGNC:29670,protein_coding,YES,NM_024312.5,-,1,P1,CCDS9088.1,ENSP00000299314,Q3T906.151,-,UPI000004CC0C,Q3T906-1,-,Ensembl,-,-,-,-,1,-,-,-,12/20,-,-,ENST00000299314.12:c.1613-25dup,-,-,0.002679,0.000708,0.000000,0.000663,0.000000,0.001548,0.006169,0.0000,0.004177,0.001461,0.001681,0.01207,gnomADe_FIN,"likely_benign,likely_pathogenic",-,1,-,-,-,-,-,-,chr12,101765328,NaN,12.0,12,True,NaN,NaN,NaN,NaN,NaN,False,True,True,False,Less common (<10%),Rare (<5%),True,False,True,PATHO
13912,chr1_11794419_T/G,chr1:11794419,G,ENSG00000215910,ENST00000688073,Transcript,downstream_gene_variant,NaN,NaN,NaN,-,-,"rs1801131,CM981315,COSV57171183",P931263,HE

In [21]:
# create a new column 'Database' in both dataframes, if MAX.AF is not NA and ClinVar is NA label 'gnomAD', if ClinVar is not NA and MAX.AF is NA label 'ClinVar', if both are not NA label 'gnomAD & ClinVar', else label 'No DB source'
cad12['Database'] = np.select([cad12['ClinVar'].notna() & cad12['MAX.AF'].isna(), 
                                  cad12['MAX.AF'].notna() & cad12['ClinVar'].isna(), 
                                  cad12['MAX.AF'].notna() & cad12['ClinVar'].notna(), 
                                  cad12['MAX.AF'].isna() & cad12['ClinVar'].isna()], 
                                  ['ClinVar', 'gnomAD', 'gnomAD & ClinVar', 'No DB source'], 
                                  default = np.nan)
checkColumnValue(cad12, 'Database')

,Count,count
0,gnomAD,3428264
1,No DB source,35778
2,gnomAD & ClinVar,32158
3,ClinVar,1


In [22]:
cad63['Database'] = np.select([cad63['ClinVar'].notna() & cad63['MAX.AF'].isna(), 
                                  cad63['MAX.AF'].notna() & cad63['ClinVar'].isna(), 
                                  cad63['MAX.AF'].notna() & cad63['ClinVar'].notna(), 
                                  cad63['MAX.AF'].isna() & cad63['ClinVar'].isna()], 
                                  ['ClinVar', 'gnomAD', 'gnomAD & ClinVar', 'No DB source'], 
                                  default = np.nan)
checkColumnValue(cad63, 'Database')

,Count,count
0,gnomAD,3641404
1,gnomAD & ClinVar,33479
2,No DB source,26131


In [23]:
print(cad12.shape)
print(cad63.shape)

(3496201, 96)
(3701014, 96)


In [24]:
cad12.to_csv('4-cad12-snv-indel-3496201.csv', index=False)
cad63.to_csv('4-cad63-snv-indel-3701014.csv', index=False)

In [25]:
crosstabit(cad12, 'MAF', 'Database', True)

Database,ClinVar,No DB source,gnomAD,gnomAD & ClinVar,Total
MAF,,,,,
Common (>=10%),0,0,3221872,31160,3253032
Less common (<10%),0,0,152004,856,152860
Rare (<1%),0,0,54388,142,54530
nan,1,35778,0,0,35779
Total,1,35778,3428264,32158,3496201


In [26]:
crosstabit(cad63, 'MAF', 'Database', True)




# TODO 
# 1. note the clinvar col with labels is not star reviewed, check the clin.sig column to confirm high confidence pathogenicity
# 2. added Database column here instead of in R to keep all label functions together in python

Database,No DB source,gnomAD,gnomAD & ClinVar,Total
MAF,,,,
Common (>=10%),0,3430130,32339,3462469
Less common (<10%),0,156632,992,157624
Rare (<1%),0,54642,148,54790
nan,26131,0,0,26131
Total,26131,3641404,33479,3701014


In [1]:
checkColumnValue(cad12, 'var.in.both.lines')

NameError: name 'checkColumnValue' is not defined